In [1]:
pip install --upgrade transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 13.1 MB/s eta 0:00:00


In [2]:
!pip uninstall -y huggingface_hub
!pip install -U huggingface_hub

Found existing installation: huggingface-hub 0.35.3
Uninstalling huggingface-hub-0.35.3:
  Successfully uninstalled huggingface-hub-0.35.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 14.4 MB/s eta 0:00:00


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = 'meta-llama/Llama-3.2-1B-Instruct'

# Quantization process

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # load model weights in 4-bit float instead of BF16
    bnb_4bit_use_double_quant=True, # quantize two times
    bnb_4bit_quant_type="nf4", # NormalFloat 4-bit
    bnb_4bit_compute_dtype="bfloat16" # When forward or backward, transform back to BF16 to achive higher compute precision
)

quantized_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": torch.cuda.current_device()}
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Testing model before finetuning

message = [
    {"role":"system","content":"You are a helpful assistant."},
    {"role":"user","content":"Render a 3D model of a house."}
]

inputs = tokenizer.apply_chat_template(
    message,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to(device)

quantized_model.to(device)
outputs = quantized_model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


I can provide a simple 3D model of a house using text-based representations. Please note that I'll be using ASCII characters for this purpose.

Here's a basic 3D model of a


In [ ]:
from datasets import load_dataset
# Loading alpaca dataset
raw_datasets = load_dataset("tatsu-lab/alpaca",split='train')

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [ ]:
# The length of alpaca dataset is about 52000 samples with train part, it is too large for the free GPU on colab or kaggle so I randomly choose 15000 samples in the dataset 
raw_datasets = raw_datasets.shuffle(seed=42)
train_datasets = raw_datasets.select(range(15000))

In [ ]:
# Setting pad_token for tokenizer as the llama tokenizer does not have it
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def create_format_input(example):
    # Concatenate input text with prompt
    full_text = example["instruction"]
    if example["input"]:
        full_text += "\nInput: " + example["input"]

    # Making prompt
    messages = [
        {"role": "user", "content": full_text},
        {"role": "assistant", "content": example["output"]}
    ]

    # Tokenize with chat template
    # Chat template will apply template for the whole chat history
    tokenized = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        max_length=512,
        truncation=True,
        add_generation_prompt=False,  
    )

    # Return the right format
    return {
        "input_ids": tokenized,
        "labels": tokenized.copy() 
    }


In [12]:
tokenized_datasets = train_datasets.map(create_format_input, remove_columns=train_datasets.column_names)

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

In [ ]:
from peft import LoraConfig, get_peft_model

# Setting the LoraConfig and apply it to model

lora_config = LoraConfig(
    r=16, # r is one of the dimensions of A,B matrix when we decomposite a layer in the model
    lora_alpha=32, # lora_alpha is scaling factor that determine how strong the adapter affect to model 
    target_modules=['q_proj','v_proj'] # target modules to apply lora is querry and value matrix,
    lora_dropout=0.05,
    task_type='CAUSAL_LM'
)

lora_model = get_peft_model(quantized_model,lora_config)

In [14]:
!pip install -U trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 13.4 MB/s eta 0:00:00


In [ ]:
# import Supervised-Fine-Tuning Trainer and Supervised-Fine-Tuning Config

from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments

training_args = SFTConfig(
    output_dir='./qlora_llama3.2',
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    fp16=True,
    learning_rate=2e-4,
    logging_steps=100,
    save_strategy='epoch',
    weight_decay=0.0,
    eval_strategy='no',
    report_to='none'
)

trainer = SFTTrainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_datasets,
    processing_class=tokenizer,
)

In [18]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=469, training_loss=1.2842862712803171, metrics={'train_runtime': 1900.6919, 'train_samples_per_second': 7.892, 'train_steps_per_second': 0.247, 'total_flos': 1.7474397678010368e+16, 'train_loss': 1.2842862712803171, 'entropy': 1.2552901014328004, 'num_tokens': 1633607.0, 'mean_token_accuracy': 0.7102084483464559, 'epoch': 1.0})

In [ ]:
from peft import PeftModel

# Loading another model to apply the trained adapter

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16"
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map='auto'
)

peft_model = PeftModel.from_pretrained(base_model,"/content/qlora_llama3.2/checkpoint-469")

In [ ]:
# Testing model with trained adapter

message = [
    {"role":"system","content":"You are a helpful assistant."},
    {"role":"user","content":"Render a 3D model of a house."}
]

inputs = tokenizer.apply_chat_template(
    message,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to(device)

peft_model.to(device)
outputs = peft_model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


This is a 3D model of a house. It has a rectangular body with a front door, two windows, a roof, and a chimney. The front door is made of wood and has


In [ ]:
import shutil
from pathlib import Path

# making the tree path to save neccessary file to deploy with gradio on hugging face space

llama_chat_demo_path = Path("demos/llama_chat/")

if llama_chat_demo_path.exists():
  shutil.rmtree(llama_chat_demo_path)
llama_chat_demo_path.mkdir(parents=True,exist_ok=True) # parents=True make parents dir if they are not exist yet

!ls demos/llama_chat/

In [ ]:
# create file to save adapter

adp_pth = llama_chat_demo_path/"adapter"
adp_pth.mkdir(parents=True,exist_ok=True)

In [ ]:
adapter_config = "/content/qlora_llama3.2/checkpoint-469/adapter_config.json"
adapter_model = "/content/qlora_llama3.2/checkpoint-469/adapter_model.safetensors"
adapter_des = llama_chat_demo_path / "adapter"
shutil.copy2(src=adapter_config,dst=adapter_des)
shutil.copy2(src=adapter_model,dst=adapter_des)

'demos/llama_chat/adapter.pth/adapter_model.safetensors'

In [ ]:
%%writefile demos/llama_chat/model.py

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# This file will create a new quantized llama model

def create_quantized_llama():
    model_name = "meta-llama/Llama-3.2-1B-Instruct"

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype="bfloat16"
    )

    quantized_model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto"
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    return quantized_model, tokenizer

Writing demos/llama_chat/model.py


In [ ]:
import gradio as gr
import os
import torch

from model import create_quantized_llama
from timeit import default_timer as timer
from typing import Tuple, Dict
from peft import PeftModel


# Get token to use llama
from huggingface_hub import login
import os

login(token=os.getenv("HF_TOKEN"))  


model, tokenizer = create_quantized_llama()

peft_model = PeftModel.from_pretrained(model,"adapter")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
peft_model.to(device)

history = [] # saving chat history

def response(text):
    global history
    
    start_time = timer()

    messages = [{"role":"system","content":"You are a helpful assistant."}]

    for user, bot in history:
        messages.append({"role":"user","content":user})
        messages.append({"role":"assistant","content":bot})
    messages.append({"role":"user","content":text})
    
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt"
    ).to(device)
    
    outputs = peft_model.generate(**inputs, max_new_tokens=100)
    running_time = round(timer() - start_time,5)
    model_response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    history.append((text, model_response))
    return history, "" 
    
examples = [
    "What is the capital of France?",
    "How did Julius Caeser die?",
    "Given two examples of a liquid."
]

with gr.Blocks() as demo:
    gr.Markdown("# LLaMA 3.2 Chatbot 🤖") 
    gr.Markdown("A model based on LLaMA 3.2 1B instruct")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Type a message...")
    send_btn = gr.Button("Send")

    for ex in examples:
        btn = gr.Button(ex)
        btn.click(lambda x=ex: response(x), inputs=None, outputs=[chatbot, msg])
    
    send_btn.click(response, inputs=msg, outputs=[chatbot, msg])
    msg.submit(response, inputs=msg, outputs=[chatbot, msg])
    
demo.launch()

Writing demos/llama_chat/app.py


In [32]:
%%writefile demos/llama_chat/requirements.txt
torch
transformers
gradio
peft
bitsandbytes
accelerate
trl

Writing demos/llama_chat/requirements.txt


In [33]:
!cd demos/llama_chat && zip -r ../llama_chat.zip * -x "*.pyc" "*.ipynb" "*__pycache__*" "*ipynb_checkpoints*"

  adding: adapter.pth/ (stored 0%)
  adding: adapter.pth/adapter_model.safetensors (deflated 8%)
  adding: adapter.pth/adapter_config.json (deflated 55%)
  adding: app.py (deflated 53%)
  adding: model.py (deflated 52%)
  adding: requirements.txt (deflated 8%)


In [35]:
model_name = 'meta-llama/Llama-3.2-1B-Instruct'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16"
)

quantized_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": torch.cuda.current_device()}
)

quantized_model.save_pretrained("/content/quantized_model", safe_serialization=True)

In [ ]:
import os

def get_folder_size(path):
    total = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total += os.path.getsize(fp)
    return total

quantized_model_size = get_folder_size("/content/quantized_model")
adapter_size = get_folder_size("/content/qlora_llama3.2/checkpoint-469")

print(f"Quantized model size: {quantized_model_size / (1024**3):.2f} GB")
print(f"LoRA adapter size: {adapter_size / (1024**2):.2f} MB")


Quantized model size: 0.96 GB
LoRA adapter size: 36.06 MB


+ We can see that the size of quantized model plus adapter is about 0.97 GB 
+ the base model (Llama) is heavier than it many times, Llama 3.2 1B instruct is about 4.6 GB if we run the cell under

In [ ]:
model_name = 'meta-llama/Llama-3.2-1B-Instruct'

based_model = AutoModelForCausalLM.from_pretrained(
    model_name
)

based_model.save_pretrained("/content/based_model", safe_serialization=True)

In [ ]:

base_size = get_folder_size("/kaggle/working/based_model")

print(f"Based model size: {base_size / (1024**3):.2f} GB")